### <font color = yellow> This is a clean copy of fifo_rows_redo.ipynb at creation point

In [ ]:
library(tibble)

In [ ]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

In [ ]:
options(pillar.width = Inf)


## <font color = yellow> create_df function () - reads .csv

In [ ]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [ ]:
head(df,2)

In [ ]:
df$flag <- FALSE
df$true_false <- FALSE


In [ ]:
head(df, 2)

## <font color = yellow>1. Create data & dfs

In [5]:
create_data <- function () {   
   
    df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
    df$date <- as.Date(df$date, origin = "1899-12-30")
 
    df$flag <- FALSE
    df$true_false <- FALSE
    
    sale_records <- data.frame() 

    # create dfs from filtered df
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    b <- buys[1,]
    s <- sells[1,]
    
    
    # Assign variables to the global environment
    assign("df", df, envir = .GlobalEnv)
    assign("buys", buys, envir = .GlobalEnv)
    assign("sells", sells, envir = .GlobalEnv)
    assign("b", b, envir = .GlobalEnv)
    assign("s", s, envir = .GlobalEnv)
    
    }

# create_data()
# sale_records <- data.frame()

In [ ]:
dim(df)


In [ ]:
sale_records <- data.frame()

## <font color = lime> Examining data and dfs

In [ ]:
nrow(buys)
nrow(sells)
dim(sells)
head(buys, 3)
head(sells,3)

In [ ]:
dim(b)
b

### <font color = yellow>2. update_b() - updates the basic b variable with data from buy/sell rows 
    
#### b>s, b<s, b==s,

In [ ]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

In [ ]:
last_row <- tail(buys, n = 1)
last_row

#### <font color = lime> Basic version: when b == s ( creates no c/f)
    
#### <font color = lime> Basic version: when b > s (creates b c/f)    
    
#### <font color = lime> Basic version: when b < s (creates s c/f)
    

In [7]:
# # update b based upon the Sale Type: b>s, b<s, b==s 

update_b <- function(b,s, buys) {


        # ( b > s ) ,  Buy C/F 
    
        if ( b$quan > s$quan) { 
            
            b$sale_quan  <- s$quan    
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$sold_cf_quan <- (b$quan - s$quan)        
            b$sold_cf_pps <- b$pps
            b$sold_cf_basis <- b$sold_cf_quan * b$pps 
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has buy c/f 
            b$flag <- TRUE 
            buys$flag[1] <- TRUE
            
            
            # All s will be eliminated (Sold) in this transaction
            s$type <- "SOLD"
            cat("POC - updated_s when b > s: ", s$type, "\n")
    
    
       # ( s > b ), has sell carryforward
 
        }else if(b$quan < s$quan) {
            
            b$basis <-  b$quan * b$pps
            b$sale_quan <- b$quan
           
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$basis
         
            # Sell c/f
            sold_cf_quan <- s$quan - b$quan           
            b$sold_cf_quan <- sold_cf_quan
            b$sold_cf_pps <- s$pps
            b$sold_cf_basis <- sold_cf_quan * s$pps
                   
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has sell c/f 
            b$true_false <- TRUE
           
 #  s$type <- "SELL-CF"   
            
        }else if (b$quan == s$quan) {   
            cat("proof 3 \n")
                  
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$buy_quan * b$pps

            b$sell_quan <- s$quan    
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps
            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # need a new flag setting for update_buy_sells fct
            if(b$type == "Buy CF" && s$type == "Sell CF") {
        cat("proof \n")
                b$co <- "CFCF"
                b$true_false <- TRUE
                s$tru_false <- TRUE
                } 
        
            s$type <- "EQ SOLD"
            cat("POC - updated_s when b == s: ", s$type, "\n")            
            }
            
        # s$type <- "SOLD"
        # cat("POC - updated_s : ", s$type, "\n")
        return(list(b,s, buys)) 
     
  }  # fct

# updated_b_list <- update_b(b,s,buys)

# updated_b <- updated_b_list[[1]]
# updated_s <- updated_b_list[[2]]
# buys <- updated_b_list[[3]]

# dim(updated_b)
# dim(buys)
    

In [ ]:
updated_b_list <- update_b(b,s,buys)

updated_b <- updated_b_list[[1]]
updated_s <- updated_b_list[[2]]
buys <- updated_b_list[[3]]

dim(updated_b)
dim(buys)
    

In [ ]:
updated_s
updated_b
b[1,]

In [ ]:
updated_b

### <font color = yellow>3. create new sales_records (rbind) with the updated_b 

In [ ]:
# df_name <- "iris"  # Replace with your data frame name
# tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)
         

In [ ]:
# df_name <- "sale_records"
# tryCatch(is.data.frame(get(df_name)), error = function(cond) FALSE)

In [ ]:
# if (exists("sale_records") && is.data.frame(get("sale_records"))) {
#   # Data frame exists and is a valid data frame
#   # Your code here...
#     cat("sale_records exists:     ", "\n") 
# } else {
#   # Data frame does not exist or is not a valid data frame
#   # Handle the case where the data frame is missing...
#     cat("Newly created sale_records with updated_b data. ", "\n")
#     sale_records <- data.frame()
#     sale_records <- rbind(sale_records, updated_b)    
#     # sale_records <- add_row(updated_b ) # , .before = 1)
# }
# sale_records
        
        

In [ ]:
# First go around needs rbind()
sale_records <- rbind(sale_records, updated_b)    

### <font color = yellow> 4. update_buys_sells() 
### <font color = yellow>  update buys or sells (add_row) when c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction


#### <font color = lime> How it's done

In [ ]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

In [ ]:
sale_records

In [11]:
# Evaluate the type of carryforward, then update buys or sells w/cf to prepare for next transaction (loop)

update_buys_sells <- function(buys, sells) {

    #  Check the most recent row ??? of sale_record or the last row of sale_record (was bind() or add_row() ? )
    
   last_row <- head(sale_records, n = 1)
   
    #first_row <- head(sale_records, n=1)
    
    # b == s (no c/f)
    if(last_row$flag == FALSE && last_row$true_false == FALSE) {
         cat("\n", "The buy == sell, t/f No update to buys or sells.  NO carryforward. ", last_row$buy_cf_quan, "\n")
      }  
        
    # both start with the same inputs from the sale_records
    b1 <- last_row
    s1 <- last_row
    
    
    
    #  b > s  ... buy c/f will be created 
   
    if (last_row$flag == TRUE && last_row$true_false == FALSE ) {
        
        
        cat("\n", "There is Buy carryforward. Update buys ", last_row$sold_cf_quan, "\n")

        last_row$co <- "c/f"
       
        
        
   
        
        
# -------------------------------------------------------------------------------------------------------------------------   
        
        
        # NEW  ... This section makes changes in sells[2, ]  which becomes sells[3,] , due to add_row()
                
        
        s2cf_quan <- sells$quan[2] - s1$sold_cf_quan
        s2cf_pps <- sells$pps[2]
        
        cat("the s2cf_quan is: ", s2cf_quan, "\n")
        cat("the s2cf_pps is ", s2cf_pps, "\n")
        
        sells$quan[2] <- s2cf_quan
        sells$pps[2] <-  s1$sell_pps                     #    s2cf_pps 
        sells[2, ]
        sells$basis[2] <- sells$pps[2] * s2cf_quan
        
        
       
        
# -------------------------------------------------------------------------------------------------------------------------    
        
        
        #  The section updates b1   .... t/f buys
        
       
        b1$type <- "Buy CF"
        b1$quan <- b1$sold_cf_quan
        b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- TRUE
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- buys
        
      #  test <- test %>% add_row(b1, .before = 1)
        test <- test %>% add_row(b1, .after = 1)
        buys <- test
        
  # ---------------------------------------------------------------------------------------------------------------      
        
               
        # this section updates s1 ... t/f sells
       
        
            
        last_row$co <- "c/f"    
        
       
        s1$type <- "Sell CF"
        s1$quan <- s1$sold_cf_quan
        s1$pps <-  s1$sell_pps               #    s1$sold_cf_pps
        s1$basis <- s1$sell_pps
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .before = 1)
        sells <- test
        
 
        
        
# =================================================================================================================
# ==============================================================================================================
 # =================================================================================================================
# ==============================================================================================================
              
        
             #   s > b ... sell carryforward will be created    
        

    
    } else if (last_row$flag == FALSE && last_row$true_false == TRUE ) {
        
        
       
        cat("\n", "There is Sold carryback. Update sells carryback -2 ", last_row$sold_cf_quan, "\n")
           # cat("dim(s1) : ", dim(s1), "\n")
         
        last_row$co <- "c/b"      
       
        s1$type <- "Sell CB"
        s1$quan <- s1$sold_cf_quan
        s1$pps <- sells$pps[1]
        s1$basis <- s1$sold_cf_basis
        s1$sale_quan <- 0
        s1[ , 10:ncol(s1)] <- 0

        test <- sells
        test <- test %>% add_row(s1, .after = 1)
        sells <- test
 
    # =================================================================================================================
# ==============================================================================================================
       
  # =================================================================================================================
# ==============================================================================================================
       
        ## must also update the buys to match that sells c/f (this will allow for loop2 to begin 
        ## with both the buy and sell processing the c/f (example when sell cf = 900, then the 
        ## buy row needs to be that same 900 .... do not delete, just update here and now 
        
        b1$type <- "Buy CB"
        # b1$quan <- b1$sold_cf_quan
       # b1$pps <- b1$sold_cf_pps
        b1$basis <- b1$sold_cf_basis
        b1$flag <- FALSE
        
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0
        
        test2 <- buys
    #    test2 <- test2 %>% add_row(b1, .before = 1)
        buys <- test2
                      
        }

  # =================================================================================================================
# ==============================================================================================================
# =================================================================================================================
# ==============================================================================================================
         
    
    # print by type of c/f  ( buy or sell ) 
        if(last_row$flag == TRUE) {
            cat("printing updated buys[1 ], dim of buys increases with c/f ", "\n")
            b1
            #buys[1, ]
        }else if (last_row$true_false == TRUE) {
            cat("printing: s1 ... dim of sells increases with c/f ", "\n")
            s1
        }else {
            cat("If msg here, there is no CF and potential issue exists in fct update_buys_sells. ", "\n")
            }
    
    return(list(buys, sells))
     
  }  # fct

# fct call
# return_list <- update_buys_sells(buys, sells)

# buys <- return_list[[1]]
# sells <- return_list[[2]]

# dim(buys)
# dim(sells)

In [ ]:
# fct call
return_list <- update_buys_sells(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)

#### <font color = yellow>5. Snip(), delete row from buys and sells / replace row when c/f 

In [ ]:
range(x <- sort(round(stats::rnorm(10) - 1.2, 1)))

if(any(x < 0)) cat("x contains negative values\n")

In [ ]:
if(any(buys[1,] == TRUE)) cat("buys contains cf \n")

In [ ]:
buys[1:3,]
sells[1:3, ]

In [ ]:
sale_records

In [13]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]
copy_snip <- function (b, s, buys, sells) {
   
    # parameters - rep the input vars (arguments)  ... may be extraneous
    b1 <- buys [1, ]
    s1 <- sells [1, ]
    
       
    # snip .... if 
        
    if(buys$quan[1] == sells$quan[1] && buys$type[1] == "Buy CF" && sells$type[1] == "Sell CF")  { #  sells$type[1]) {
        cat("new condtional works for filtering in copy_snip when cf-cf \n")
        buys <-  buys[-1, ]
        sells <- sells[-1, ]    
     return(list(b1, s1, buys, sells))  
        
  #  if(buys$type[1] == "Buy CF")   { 
    } else if (buys$type[1] == "Buy CF" || buys$flag[1] == TRUE ) {   #  && buys$true_false == FALSE)   {         
   # if(buys$flag[1] == "TRUE")   {         
        cat(" Contains buy carryforward - remove buys c/f row and DO snip sells \n")         
         # remove the original buys and sells
         buys <- buys[-1, ]    
         sells <- sells[-2, ]
        return(list(b1, s1, buys, sells))                   
        
    } else if (sells$type[1] == "Sell CF") {
        cat("sells contains carryforward .. Replace sells and DO snip buys \n") 
        sells <- sells[-2,]
        buys <- buys[-1,]
        return(list(b1, s1, buys, sells))
# new       
    } else if (sells$type[2] == "Sell CB") {
        cat("sells contains carryback .. Remove sells[1,]", "\n") 
        sells <- sells[-1,]
        buys <- buys[-1,]
        return(list(b1, s1, buys, sells))
        
     }else if (buys$co[1] == "CFCF" ) {
        cat("this is a cf/cf wash sales ... replace buys[1,] sells[1,] \n")
        buys <- buys[-1]
        sells <- sells[-1]
         return(list(b1, s1, buys, sells))
        
    } else if(b1$flag == FALSE & b1$true_false == FALSE) {
        
        cat("check false / false \n")        
        buys <- buys[-1,]
        sells <- sells[-1,]
        
        return(list(b1, s1, buys, sells))
        
    } else {
        cat("There is no carryforward in buys or sells, This msgs means there is an issue. \n")
        # buys <- buys[-1,]
        # sells <- sells[-1,]
        return(list(b1, s1, buys, sells))    
    }
 
 } # fct     
   
# fct call passing 2 arguments
# copy_snip_list <- copy_snip(b, s, buys, sells) 

# b1 <- copy_snip_list[[1]]
# s1 <- copy_snip_list[[2]]
# buys <- copy_snip_list[[3]]
# sells <- copy_snip_list[[4]]

# # after
# dim(buys)
# dim(sells)
# # b1
# s1

In [ ]:
copy_snip_list <- copy_snip(b, s, buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
# b1
# s1

In [ ]:
b1

#### <font color = lime> QC: Loop 1 review buys & sells

In [ ]:
buys[1:4,]
sells[1:4,]

#### <font color = lime> QC: Loop 1 review sale_record for accuracy

In [ ]:
sale_records

### <font color = lime> Got it!  The components above are modular and can be combined into a loop

In [ ]:
if (exists("sale_records") && is.data.frame(get("sale_records"))) {
  # Data frame exists and is a valid data frame
  # Your code here...
    cat("sale_records exists:     ", "\n") 
} 

In [ ]:
sale_records

In [ ]:
# stopps #  Old code

# <font color = cyan> Proof of Concept above. End of loop 1 

#### <font color = yellow>1. Create data & dfs

In [ ]:
b <- buys[1,]
s <- sells[1,]

In [ ]:
b
s

### <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [ ]:
updated_b_list <- update_b(b,s,buys)

updated_b <- updated_b_list[[1]]
updated_s <- updated_b_list[[2]]
buys <- updated_b_list[[3]]

dim(updated_b)
dim

### <font color = yellow>3. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [ ]:
b

In [ ]:
sale_records <- sale_records %>% add_row(updated_b , .before = 1)
sale_records[1,]
sale_records

## <font color = lime> CAUTION: Step3 using either before/after, will effect Step 4 below:  

In [ ]:
buys[1:4,]
sells[1:4,]

#### <font color = yellow> 4. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction

In [ ]:
# # fct call
return_list <- update_buys_sells(buys, sells)

buys <- return_list[[1]]
sells <- return_list[[2]]

dim(buys)
dim(sells)


#### <font color = yellow>5. snips () Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )

In [ ]:
copy_snip_list <- copy_snip(b, s, buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
dim(buys)
dim(sells)
# b1
s1

#### <font color = lime> QC: Loop 2 review sale_record for accuracy

In [ ]:
sale_records

#### <font color = lime> QC: Loop 2 review buys & sells

In [ ]:
dim(buys)
dim(sells)
head(buys,4)
head(sells,4)

In [ ]:
# stopps

# <font color = cyan> Proof of Concept above. End of loop 2

#### <font color = yellow>1. Create data & dfs

In [ ]:
b <- buys[1,]
s <- sells[1,]

### <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [ ]:
updated_b_list <- update_b(b,s,buys)

updated_b <- updated_b_list[[1]]
updated_s <- updated_b_list[[2]]
buys <- updated_b_list[[3]]

# dim(updated_b)
# dim

### <font color = yellow>3. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [ ]:
sale_records <- sale_records %>% add_row(updated_b , .before = 1)
# sale_records[1,]
# sale_records

#### <font color = yellow> 4. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for

In [ ]:
# # fct call
return_list <- update_buys_sells(buys, sells)
buys <- return_list[[1]]
sells <- return_list[[2]]

# dim(buys)
# dim(sells)


#### <font color = yellow>5. snips () Create a new variable from buys/sells row 1, then delete row 1 from buys and sells (copy_snip () )

In [ ]:
copy_snip_list <- copy_snip(b, s, buys, sells) 

b1 <- copy_snip_list[[1]]
s1 <- copy_snip_list[[2]]
buys <- copy_snip_list[[3]]
sells <- copy_snip_list[[4]]

# after
# dim(buys)
# dim(sells)
# # b1
# s1

#### <font color = lime> QC: Loop 3 review buys & sells
    


In [ ]:
dim(buys)
dim(sells)
head(buys, 5)
head(sells, 5)

#### <font color = lime> QC: Loop 2 review sale_record for accuracy

In [ ]:
sale_records

# <font color = cyan> Proof of Concept above. End of loop 3

In [4]:
stopps

ERROR: Error in eval(expr, envir, enclos): object 'stopps' not found


In [15]:
library(tibble)
create_data()
sale_records <- data.frame()
   

for(i in 1:nrow(buys)) {
    
    b <- buys[1,]
    s <- sells[1,]
    
    # fct call 2
    updated_b_list <- update_b(b,s,buys)
    updated_b <- updated_b_list[[1]]
    updated_s <- updated_b_list[[2]]
    buys <- updated_b_list[[3]]
    
    # fct call 
    if(i == 1) {
    sale_records <- rbind(sale_records, updated_b)    
        
    } else {sale_records <- sale_records %>% add_row(updated_b , .before = 1) }
    
    # fct call 4
    return_list <- update_buys_sells(buys, sells)
    buys <- return_list[[1]]
    sells <- return_list[[2]]
    # fct call 6
    copy_snip_list <- copy_snip(b, s, buys, sells) 
    b1 <- copy_snip_list[[1]]
    s1 <- copy_snip_list[[2]]
    buys <- copy_snip_list[[3]]
    sells <- copy_snip_list[[4]]
    
    }
                            
sale_records

POC - updated_s when b > s:  SOLD 

 There is Buy carryforward. Update buys  50 
the s2cf_quan is:  950 
the s2cf_pps is  6.68255 
printing updated buys[1 ], dim of buys increases with c/f  
 Contains buy carryforward - remove buys c/f row and DO snip sells 
proof 3 
proof 
POC - updated_s when b == s:  EQ SOLD 
printing updated buys[1 ], dim of buys increases with c/f  
new condtional works for filtering in copy_snip when cf-cf 
POC - updated_s when b > s:  SOLD 

 There is Buy carryforward. Update buys  50 
the s2cf_quan is:  950 
the s2cf_pps is  6.255 
printing updated buys[1 ], dim of buys increases with c/f  
 Contains buy carryforward - remove buys c/f row and DO snip sells 
proof 3 
proof 
POC - updated_s when b == s:  EQ SOLD 
printing updated buys[1 ], dim of buys increases with c/f  
new condtional works for filtering in copy_snip when cf-cf 
POC - updated_s when b > s:  SOLD 

 There is Buy carryforward. Update buys  50 
the s2cf_quan is:  950 
the s2cf_pps is  6.1901 
prin

index,date,type,co,quan,pps,basis,sale_quan,flag,buy_cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sold_cf_quan,sold_cf_pps,sold_cf_basis,true_false
<int>,<date>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
24,2023-03-07,Buy,AMC,2900,6.1491,17832.39,-3000,TRUE,0,⋯,6.1491,-18447.300,-3000,5.5124,-16537.20,1910.100,5900,6.1491,36279.69,0
22,2023-03-03,Buy CF,CFCF,6000,6.4850,38910.00,0,TRUE,0,⋯,6.4850,38910.000,6000,5.5124,33074.40,-5835.600,0,0.0000,0.00,1
22,2023-03-03,Buy,AMC,1000,6.4850,6485.00,-5000,TRUE,0,⋯,6.4850,-32425.000,-5000,5.5124,-27562.00,4863.000,6000,6.4850,38910.00,0
21,2023-03-03,Buy CF,CFCF,6000,6.5891,39534.60,0,TRUE,0,⋯,6.5891,39534.600,6000,5.5124,33074.40,-6460.200,0,0.0000,0.00,1
21,2023-03-03,Buy,AMC,5000,6.5891,32945.50,-1000,TRUE,0,⋯,6.5891,-6589.100,-1000,5.5124,-5512.40,1076.700,6000,6.5891,39534.60,0
17,2023-02-15,Buy CF,CFCF,2850,4.7801,13623.28,0,TRUE,0,⋯,4.7801,13623.285,2850,5.5124,15710.34,2087.055,0,0.0000,0.00,1
17,2023-02-15,Buy,AMC,4000,4.7801,19120.40,1150,TRUE,0,⋯,4.7801,5497.115,1150,5.5124,6339.26,842.145,2850,4.7801,13623.28,0
15,2023-02-13,Buy CF,CFCF,1850,4.7100,8713.50,0,TRUE,0,⋯,4.7100,8713.500,1850,5.5124,10197.94,1484.440,0,0.0000,0.00,1
15,2023-02-13,Buy,AMC,2000,4.7100,9420.00,150,TRUE,0,⋯,4.7100,706.500,150,5.5124,826.86,120.360,1850,4.7100,8713.50,0


In [19]:
write.csv(sale_records, file = "sale_records.csv")

In [18]:
?write.csv

write.table {utils},R Documentation
x,"the object to be written, preferably a matrix or data frame. If not, it is attempted to coerce x to a data frame."
file,"either a character string naming a file or a connection open for writing. """" indicates output to the console."
append,"logical. Only relevant if file is a character string. If TRUE, the output is appended to the file. If FALSE, any existing file of the name is destroyed."
quote,"a logical value (TRUE or FALSE) or a numeric vector. If TRUE, any character or factor columns will be surrounded by double quotes. If a numeric vector, its elements are taken as the indices of columns to quote. In both cases, row and column names are quoted if they are written. If FALSE, nothing is quoted."
sep,the field separator string. Values within each row of x are separated by this string.
eol,"the character(s) to print at the end of each line (row). For example, eol = ""\r\n"" will produce Windows' line endings on a Unix-alike OS, and eol = ""\r"" will produce files as expected by Excel:mac 2004."
na,the string to use for missing values in the data.
dec,the string to use for decimal points in numeric or complex columns: must be a single character.
row.names,"either a logical value indicating whether the row names of x are to be written along with x, or a character vector of row names to be written."
col.names,"either a logical value indicating whether the column names of x are to be written along with x, or a character vector of column names to be written. See the section on ‘CSV files’ for the meaning of col.names = NA."
